In [28]:
import sys
sys.path.append('../')
from util import corpus, tokenize, util, alignment, word2vec, retrieval

from tqdm.notebook import tqdm as tq

import pandas as pd

In [11]:
# df = util.decompress_pickle('../../data/word2vec/epo/corpus/epo_18_20_df_phrasal')
df = util.decompress_pickle('../../data/epo_corpus_all_years/pre_proc/epo_10_20_df_wo_pre')
df

,name,titles,en,de
0,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,A modular power converter comprising a thermal...,"Modularer Leistungswandler, der Folgendes umfa..."
1,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,"The modular power converter of claim 1, wherei...","Modularer Leistungswandler nach Anspruch 1, wo..."
2,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,"The modular power converter of claim 1, wherei...","Modularer Leistungswandler nach Anspruch 1, wo..."
3,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,"The modular power converter of claim 1, wherei...","Modularer Leistungswandler nach Anspruch 1, wo..."
4,EP03703859NWB1.xml,de\tMODULARER LEISTUNGSWANDLER MIT FLÜSSIGGEKÜ...,"The modular power converter of claim 1, wherei...","Modularer Leistungswandler nach Anspruch 1, wo..."
...,...,...,...,...
545093,EP17000010NWB1.xml,de\tSTREULICHTBLENDE EINES BILDERFASSUNGSGERÄT...,"Lens hood according to Claim 10, characterized...","Streulichtblende nach Anspruch 10, dadurch gek..."
545094,EP17000010NWB1.xml,de\tSTREULICHTBLENDE EINES BILDERFASSUNGSGERÄT...,"Lens hood according to one of Claims 9 to 11, ...",Streulichtblende nach einem der Ansprüche 9 bi...
545095,EP17000010NWB1.xml,de\tSTREULICHTBLENDE EINES BILDERFASSUNGSGERÄT...,Lens hood according to one of the preceding cl...,Streulichtblende nach einem der vorherigen Ans...
545096,EP17000010NWB1.xml,de\tSTREULICHTBLENDE EINES BILDERFASSUNGSGERÄT...,Lens hood according to one of the preceding cl...,Streulichtblende nach einem der vorherigen Ans...


In [42]:
corpus_en = df.en.to_list()
corpus_de = df.de.to_list()

In [70]:
import spacy
from spacy.matcher import Matcher
nlp_en = spacy.load("en_core_web_lg")
nlp_de = spacy.load("de_core_news_lg")

/home/seb/.local/lib/python3.8/site-packages/spacy/util.py:1521: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [40]:
from spacy import displacy

# displacy.render(doc, style="dep")

In [41]:
nlp_en.add_pipe("merge_noun_chunks")
nlp_de.add_pipe("merge_noun_chunks")

<function spacy.pipeline.functions.merge_noun_chunks(doc: spacy.tokens.doc.Doc) -> spacy.tokens.doc.Doc>

In [78]:
spacy.prefer_gpu()
nlp_en = spacy.load("en_core_web_lg")
nlp_en.add_pipe("merge_noun_chunks")

triplets_en = []
for sent in tq(corpus_en[:50000]):
    try:
        doc = nlp_en(sent)
        for token in doc:
            if token.pos_ == 'VERB':
                subj, root, obj = '', '', ''
                root = token.text
                for child in token.children:
                    if 'subj' in child.dep_:
                        subj = child.text
                    if 'obj' in child.dep_:
                        obj = child.text
                if subj and root and obj:
                    # print('SUBJ: %s ROOT: %s OBJ: %s' % (subj, root, obj))
                    triplets_en.append(
                        {
                            'subj': subj,
                            'root': root,
                            'obj': obj
                        }
                    )
    except:
        pass

In [79]:
triplets_en_df = pd.DataFrame(triplets_en)
triplets_en_df.drop_duplicates(inplace=True, ignore_index=True)
triplets_en_df

,subj,root,obj
0,the support,defines,an electrical reference plane
1,the thermal support,includes,at least one extension
2,The modular power converter,forms,a matrix converter
3,which,has,a drive train
4,it,lost,communication
...,...,...,...
19539,the vehicle behavior estimation,means,252
19540,actual behavior detection,means,256
19541,determination,means,256
19542,the behavior estimation,means,252 estimates


In [80]:
util.compress_pickle('triplets_en_df', triplets_en_df)

In [36]:
len(list(set(triplets_df.root.to_list())))

377

In [ ]:
list(set(triplets_df.root.to_list()))

In [ ]:
OBJ_DEPS = ['oa', 'oc', 'og', 'op']
SUBJ_DEPS = ['sb', 'sbp']

In [74]:
spacy.prefer_gpu()
nlp_de = spacy.load("de_core_news_lg")
nlp_de.add_pipe("merge_noun_chunks")

triplets_de = []
for sent in tq(corpus_de[:50000]):
    try:
        doc = nlp_de(sent)
        for token in doc:
            if token.pos_ == 'VERB':
                subj, root, obj = '', '', ''
                root = token.text
                for child in token.children:
                    if child.dep_ in SUBJ_DEPS:
                        subj = child.text
                    if child.dep_ in OBJ_DEPS:
                        obj = child.text
                if subj and root and obj:
                    # print('SUBJ: %s ROOT: %s OBJ: %s' % (subj, root, obj))
                    triplets_de.append(
                        {
                            'subj': subj,
                            'root': root,
                            'obj': obj
                        }
                    )
    except:
        pass

In [75]:
triplets_de_df = pd.DataFrame(triplets_de)
triplets_de_df.drop_duplicates(inplace=True, ignore_index=True)
triplets_de_df

,subj,root,obj
0,der Folgendes,umfasst,einen thermischen Träger
1,der Träger,umfasst,ein Substrat
2,Modularer Leistungswandler,durchzulassen,tragen
3,der Träger,definiert,eine elektrische Bezugsebene
4,der thermische Träger,umfasst,12 wenigstens eine Verlängerung
...,...,...,...
23046,welche,speichern,eine Information
23047,welche,hindern,daran
23048,die Prüfeinheit,prüft,die Korrektheit
23049,Abfrageerlangungsmittel,beinhaltet,undBestimmungsmittel


In [76]:
util.compress_pickle('triplets_de_df', triplets_de_df)

In [ ]:
deps = 'ROOT, ac, adc, ag, ams, app, avc, cc, cd, cj, cm, cp, cvc, da, dep, dm, ep, ju, mnr, mo, ng, nk, nmc, oa, oc, og, op, par, pd, pg, ph, pm, pnc, punct, rc, re, rs, sb, sbp, svp, uc, vo'
deps = deps.split(', ')

In [61]:
for d in deps:
    print(d, spacy.explain(d))

ROOT None
ac adpositional case marker
adc adjective component
ag genitive attribute
ams measure argument of adjective
app apposition
avc adverbial phrase component
cc coordinating conjunction
cd coordinating conjunction
cj conjunct
cm comparative conjunction
cp complementizer
cvc collocational verb construction
da dative
dep unclassified dependent
dm discourse marker
ep expletive es
ju junctor
mnr postnominal modifier
mo modifier
ng negation
nk noun kernel element
nmc numerical component
oa accusative object
oc clausal object
og genitive object
op prepositional object
par parenthetical element
pd predicate
pg phrasal genitive
ph placeholder
pm morphological particle
pnc proper noun component
punct punctuation
rc relative clause
re repeated element
rs reported speech
sb subject
sbp passivized subject (PP)
svp separable verb prefix
uc unit component
vo vocative


## Load triplets

In [81]:
triplets_en_df = util.decompress_pickle('triplets_en_df')
triplets_de_df = util.decompress_pickle('triplets_de_df')

In [141]:
from collections import Counter

def get_enitites(word, lang='en'):
    enitites = []
    triplets_df = triplets_en_df if lang == 'en' else triplets_de_df
    for subj, root, obj in zip(triplets_df.subj,
                              triplets_df.root,
                              triplets_df.obj
                              ):
        if word.lower() in subj.lower() or word in obj.lower():
            print('%s | %s | %s' %(subj, root, obj))
            enitites.extend(subj.lower().split(' '))
            enitites.extend(obj.lower().split(' '))
    return enitites
            
def get_most_common(enitites):
    comm = Counter(enitites).most_common(20)
    for c in comm:
        print(c)
        
        
def get_analogies(word, lang='en'):
    enitites = get_enitites(word, lang=lang)
    get_most_common(enitites)

In [142]:
get_analogies(word='braking', lang='en')

the provided second information item | comprises | a braking torque information item
which | generates | a braking torque
the vehicle | comprises | a braking mechanism
the braking mechanism | comprises | a hydraulic disk brake
the braking mechanism | comprises | a regenerative brake
a brake manipulation portion 2 wheel braking | means | 4
the wheel braking | means | 4 a normally-closed third solenoid open close valve V3
the wheel braking | means | the interruption
An aircraft braking controller | required | 503 an output
An aircraft braking controller | required | an output
 wheel braking commander | has | an input
calculator | calculates | the target braking length
The braking device | means | 4
82 | comprises | a braking device
the safety braking instructions | means | 10 torque values
the braking characteristic curve | has | a plurality
the controller | comprises | a braking actuator
the braking actuator | comprises | a stepper motor
the motor | drives | a braking mode
the controlle

In [143]:
get_analogies(word='steuerung', lang='de')

die Steuereinrichtung | unterbindet | eine Steuerung
die Steuereinrichtung | ausführt | eine Steuerung
die Steuerung | eingerichtet | istdas Ausgangssignal
die Steuerung | aufweist | eine Berechnungsvorrichtung
eine Lastbegrenzungssteuerungseinrichtung | aufweist | durcheine Begrenzungsstarttemperaturbestimmungseinrichtung 43a
die Leistungsübertragungseinheit | aufweist | eine Zusammenwirkungssteuerungseinrichtung
die Leistungsübertragungseinheit | aufweist | eine Geschwindigkeitsänderungssteuerungsabänderungseinrichtung
die | abändert | einen Steuerungsinhalt
die Sendesteuerungseinrichtungen | enthalten | einen Beschleunigungssensor
das | beinhaltet | Annullierungssteuerung
die Steuerung | befehligt | den bordeigenen Sender
die Steuerungseinrichtung | erhält | eine Solltemperatur
die Steuerungseinrichtung | erhält | die Antriebskraft
die Steuerungseinrichtung | bestimmt | 23 eine Fahrneigung
die Fahrzeugbewegungssteuerung | steuert | einen Betrieb
die | unterstützt | die Steuerungsbet

In [144]:
get_analogies(word='combustion', lang='en')

that | control | the internal combustion engine
the first control unit | controls | the internal combustion engine
the locomotion source | comprises | an internal combustion engine
The automotive human machine interface | comprises | a vehicle internal combustion engine cranking function
  | said | combustion engine
combustion start control | means | 70
which | includes | an internal combustion engine
the first drive system | includes | an internal combustion engine
Internal combustion engine | comprising | an engine block
that | controls | the internal combustion engine
The method | operating | the hybrid internal combustion engine and air motor system
the hybrid internal combustion engine and air motor system | comprises | a an air inlet line
Method | comprises | at least one internal combustion engine
Drive | comprises | at least one internal combustion engine
internal combustion engine drives | said | at least second wheel
An internal combustion engine | characterized | inthat
The 